<a href="https://colab.research.google.com/github/codebybishwa/Grade_generator/blob/main/Grading_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
df = pd.read_excel('/content/input.xlsx', header=None)

In [97]:
# Extract metadata rows for column names, max marks, and weightage
column_names = df.iloc[0].values  # First row as column names
max_marks = df.iloc[1].values     # Second row as max marks
weightage = df.iloc[2].values     # Third row as weightage

In [98]:
weightage

array(['Weightage', nan, 30, 40, 15, 15], dtype=object)

In [99]:
# Rename columns using column_names but keep all rows in df
df.columns = column_names

# Keep metadata rows (first three rows) intact, and get student data (from row 4 onward)
student_data = df.loc[3:].copy()  # Copying to avoid modifying the original metadata

# Identify the relevant columns containing marks (from the third column onward)
marks_columns = df.columns[2:]  # Skipping the first two columns (Roll and Name)

In [100]:
marks_columns
student_data

,Roll,Name,MidSem,Endsem,Quiz 1,Quiz 2
3,1404AI01,Skand Gupta,59,87,24,36
4,1404AI02,Pankaj Kumar,85,122,21,31
5,1404AI04,Jaishree Mayank,80,139,17,9
6,1404AI07,Vaibhav Sisodiya,75,73,30,49
7,1404AI08,Bhavit Sharma,62,42,31,50
...,...,...,...,...,...,...
100,1404CS93,Aman Kumar,97,58,42,58
101,1404CS94,Sanjeet Kumar Nayak,100,160,50,75
102,1404CS95,Vivek Nakum,76,124,40,74
103,1404CS96,Akash Yada,81,121,48,29


In [101]:
# Function to calculate total marks for each student based on scaled marks
def calculate_total_marks(row):
    total = 0
    for i, col in enumerate(marks_columns, start=2):  # Start index from 2 to align with max_marks, weightage
        if pd.notnull(row[col]) and pd.notnull(max_marks[i]) and pd.notnull(weightage[i]):
            scaled_mark = (row[col] / max_marks[i]) * weightage[i]
            total += scaled_mark
    return total

In [102]:
# Apply the function to each student row and add the result as a new column
student_data['total scaled/100'] = student_data.apply(calculate_total_marks, axis=1)

In [103]:
# Define the function to assign grades based on the schema
def assign_grades(df, schema):
    total_students = len(df)
    grade_boundaries = {}

    # Calculate the number of students for each grade
    boundary = 0
    for grade, percentage in schema.items():
        count_for_grade = (percentage / 100) * total_students
        grade_boundaries[grade] = (boundary, boundary + count_for_grade)
        boundary += count_for_grade

    # Sort students by total marks to assign grades
    df_sorted = df.sort_values(by='total scaled/100', ascending=False).reset_index(drop=True)

    # Assign grades based on grade boundaries
    grade_labels = []
    for i in range(total_students):
        for grade, (lower, upper) in grade_boundaries.items():
            if lower <= i < upper:
                grade_labels.append(grade)
                break

    df_sorted['Grade'] = grade_labels
    return df_sorted

In [104]:
# Ask user if they want to continue with the default schema or provide their own
user_choice = input("Do you want to continue with the default grading schema? (yes/no): ").strip().lower()

default_schema = {
    'AA': 5,
    'AB': 15,
    'BB': 25,
    'BC': 30,
    'CC': 15,
    'CD': 5,
    'DD': 5,
}

Do you want to continue with the default grading schema? (yes/no): yes


In [105]:
# If the user chooses 'yes', apply the default schema
if user_choice == 'yes':
    student_data = assign_grades(student_data, default_schema)
else:
    # If 'no', ask user to input custom percentage values for each grade
    custom_schema = {}
    grades = ['AA', 'AB', 'BB', 'BC', 'CC', 'CD', 'DD']
    total_percentage = 0

    print("Please input the percentage for each grade (must add up to 100%):")
    for grade in grades:
        while True:
            try:
                percentage = float(input(f"Enter percentage for grade {grade}: "))
                if percentage < 0 or percentage > 100:
                    print("Percentage should be between 0 and 100.")
                    continue
                custom_schema[grade] = percentage
                total_percentage += percentage
                break
            except ValueError:
                print("Please enter a valid number.")

    if total_percentage != 100:
        print(f"Total percentage entered is {total_percentage}%. Please ensure the total equals 100%.")
    else:
        student_data = assign_grades(student_data, custom_schema)

In [106]:
# Concatenate metadata rows with student data containing calculated grades
df_with_grades = pd.concat([df.iloc[:3], student_data], ignore_index=True)

In [107]:
# Additional grades to include in the summary even if zero
all_grades = ['AA', 'AB', 'BB', 'BC', 'CC', 'CD', 'DD', 'F', 'I', 'PP', 'NP']

In [108]:
# Generate summary table with grade distribution
total_students = len(df_with_grades)
summary_data = []

# All possible grades, including additional grades with zero counts
all_grades = ['AA', 'AB', 'BB', 'BC', 'CC', 'CD', 'DD', 'F', 'I', 'PP', 'NP']

for grade in all_grades:
    percentage = default_schema.get(grade, 0)  # Use 0 for grades not in the default schema
    counts = (percentage / 100) * total_students
    rounded_counts = round(counts)
    verified_counts = df_with_grades['Grade'].value_counts().get(grade, 0)
    summary_data.append([grade, percentage, counts, rounded_counts, verified_counts])

# Create the summary table as a DataFrame
summary_df = pd.DataFrame(summary_data, columns=['Grade', 'Old IAPC Reco', 'Counts', 'Round', 'Count Verified'])


summary_df.loc[-1] = ['Total Students', total_students, '', '', '']
summary_df.index = summary_df.index + 1  # Shift index
summary_df = summary_df.sort_index()  # Sort by index to place 'Total Students' at the top

# Concatenate the summary table to the main DataFrame with an empty column in between
df_with_grades[''] = ""
df_with_summary = pd.concat([df_with_grades, summary_df], axis=1)

In [110]:
output_path = 'output.xlsx'
df_with_summary.to_excel(output_path, index=False)